<a href="https://colab.research.google.com/github/AlkimiaSoft/AIColabSamples/blob/main/GenerateIntermediateStepsAnimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install required libraries
!pip install diffusers accelerate


In [ ]:
#@title Create a Stable diffusion pipeline
from diffusers import DiffusionPipeline, EulerAncestralDiscreteScheduler
import torch

pipe = DiffusionPipeline .from_pretrained('tensor-diffusion/Dark-Sushi-2.5D',
    torch_dtype=torch.float16
)
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")

In [3]:
#@title Create a callback function to generate intermediate images from steps
vae = pipe.vae
images = []

def latents_callback(pipe, step_index, timestep, callback_kwargs):
    latents = callback_kwargs["latents"]
    latents = 1 / vae.config.scaling_factor * latents
    image = vae.decode(latents).sample[0]
    image = (image / 2 + 0.5).clamp(0, 1)
    image = image.cpu().permute(1, 2, 0).numpy()
    images.extend(pipe.numpy_to_pil(image))
    return callback_kwargs


In [6]:
#@title Generate an image calling our callback funcion in every step of the diffusion
prompt = "masterpiece, best best quality, (1girl), woman, slim body, full body, purple hair, double bun, long hair, light purple eyes, white plain t-shirt, short blue jeans"
negative_prompt = "nude, bad anatomy, blurry, fuzzy, extra legs, extra arms, extra fingers, badly drawn hands, poorly drawn feet, disfigured, out of frame, tiling, terrible art, deformed, mutated, cgi, octane, render, 3d, doll"
final_image = pipe(prompt=prompt,
                   negative_prompt = negative_prompt,
                   height=1024,
                   width=576,
                   guidance_scale=7,
                   num_inference_steps=30,
                   callback_on_step_end=latents_callback,
                   callback_on_step_end_tensor_inputs=["latents"]).images[0]


  0%|          | 0/30 [00:00<?, ?it/s]

In [7]:
from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [10]:
images.append(final_image)
image_grid(images, rows=1, cols=len(images))

In [9]:
!pip install imageio

In [11]:
import imageio
imageio.mimsave('DiffusionSteps.gif', images, duration=0.5)


In [12]:
!pip install moviepy

In [15]:
from moviepy.editor import ImageSequenceClip
import numpy as np

# Convert PIL images to NumPy arrays
image_array_np = [np.array(img) for img in images]
# Create a clip from the image sequence
clip = ImageSequenceClip(image_array_np, fps=15)

# Write the video file
clip.write_videofile("DiffusionSteps.mp4", codec="libx264")

Moviepy - Building video DiffusionSteps.mp4.
Moviepy - Writing video DiffusionSteps.mp4



Moviepy - Done !
Moviepy - video ready DiffusionSteps.mp4
